In [2]:
import multiprocessing
import time
import sys

sys.path.insert(0, '/directory/tothe/handshakefile/')
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')
import stylised_facts_data_utilities as sfd_utils
import lob_for_futures as lobfut
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from multiprocessing import Pool, freeze_support
import pickle

scaler = MinMaxScaler()
standard_scaler = StandardScaler()
import pandas as pd
import numpy as np


In [27]:


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from multiprocessing import Pool

## data files
laptop_OS_folder = '/media/ak/OS/Data/FuturesDataSemiProcessed'
LaCie_ProcessedData = '/media/ak/LaCie/ProcessedSampledData/'
# returns_data = '/media/ak/T7/August11th2022Experiments/Returns/'
t7 = '/media/ak/T71/'
t7_processed_futures_folder = os.path.join(t7, 'FuturesDataSemiProcessed')
# june_ext = os.path.join(t7, 'June4th2022Experiments')
# returns_data = [f for f in os.listdir(june_ext) if '_returns' in f]
experimentsLocation = '/media/ak/T71/August11th2022Experiments/'

# here i start with RX1 to do all the experiments in one go
symbols = os.listdir(laptop_OS_folder)
# symbol_test_folder = os.path.join(laptop_OS_folder, symbols[0])
# rx_folder = os.path.join(laptop_OS_folder, 'RX1')
du_folder = os.path.join(t7_processed_futures_folder, 'DU1')
# make this a bit more dynamic to take any function in here
files = os.listdir(du_folder)

In [26]:
os.listdir(t7_processed_futures_folder)

['VIX',
 'FB1',
 'TU1',
 'Returns',
 'RX1',
 'OE1',
 'TY1',
 'FV1',
 'JB1',
 'DU1',
 'KE1',
 'US1',
 'YM1',
 'XM1',
 'VXX',
 'G_1']

In [28]:



def produce_experiment_data(chosen_df):
    """
    # this is somewhat unit-tested in the August 7th 2022 notebook
    # function to produce dataframes for experiments
    # input: chosen df - this is a dataframe that we apply the microstructure features and the vol estimation features
    # output: experiment - dataframe with the features we want experiments for
    # written in August 2022
    # re-write it as part of the _init_ file

    """
    chosen_df_micro = lobfut.apply_micro_structure_features(chosen_df)  # get micro structure df
    vol_class = lobfut.volatilityEstimation(chosen_df)  # get the vol class

    # features I need: micro_price changes / vols /skews /etc:

    experiment_df = chosen_df_micro.loc[:, ['micro_price', 'price_imbalance',
                                            'pct_change_micro_price', 'weighted_activity_spread', ]]
    experiment_df['GK_vol'] = pd.Series(
        list(vol_class.garmanKlass(5)))  # get a sample of Garman - Klass resampled for 5 clicks
    experiment_df['arrival_rates'] = pd.Series(
        vol_class.arrival_rates().reshape(vol_class.arrival_rates().shape[0], )).replace([np.inf, -np.inf],
                                                                                         0).fillna(0)
    X = experiment_df.pct_change_micro_price.replace([np.inf, -np.inf], 0).values.reshape(-1, 1)
    norm_scaler = StandardScaler().fit(X)  # normalised scaling by mean and std
    min_max_scaler = MinMaxScaler().fit(X)  # min max scaling
    try:
        experiment_df['returns_normalised'] = norm_scaler.transform(
            X)  # use this format to get rid of the prior issues
        experiment_df['returns_mix_max'] = min_max_scaler.transform(X)  # get both issues of
        rs, rk = vol_class.realised_skewness_kurtosis()  # get skew/kurt

        experiment_df['skew'] = pd.Series(list(rs))
        experiment_df['kurt'] = pd.Series(list(rk))

        experiment_df['median_traded_volume'] = chosen_df_micro[['total_traded_volume_open',
                                                                 'total_traded_volume_high',
                                                                 'total_traded_volume_low',
                                                                 'total_traded_volume_close']].quantile(0.5, axis=1)
        experiment_df['jumps_test'] = pd.Series(vol_class.jumps_test(rollingWindow=5, sampling_param=0))
        experiment_df['relz_var'] = pd.Series(vol_class.realised_variance(rollingWindow=5))
        experiment_df['trip_quart'] = pd.Series(vol_class.tripower_quarticity(rollingWindow=5, sampling_param=0))

        experiment_df = experiment_df.replace([np.inf, -np.inf], 0).fillna(0)  # final clean up
    except ValueError:
        print("error")
        pass
    experiment_df = experiment_df.replace([np.inf, -np.inf], 0).fillna(0)  # final clean up

    return experiment_df


if __name__ == '__main__':
    # files_idx = 1

    def produce_and_dump(files_idx):
        """
        function that takes the index of a file, that index corresponds to a point in a list of files that are already
        aligned between trades and quotes and have been information clock processed
        then applies the extraction of features
        """

        date_idx = files[files_idx].split(".")[0]
        idx_file = os.path.join(du_folder, files[files_idx]) # the input here needs to be dynamic not du_folder or rx_folder
        tick_df = pd.read_pickle(idx_file)[date_idx]['tick']
        volume_df = pd.read_pickle(idx_file)[date_idx]['volume']
        calendar_df = pd.read_pickle(idx_file)[date_idx]['calendar']
        dollar_df = pd.read_pickle(idx_file)[date_idx]['dollar']

        output_dict = {'tick': produce_experiment_data(tick_df),
                       'volume': produce_experiment_data(volume_df),
                       'calendar': produce_experiment_data(calendar_df),
                       'dollar': produce_experiment_data(dollar_df),
                       }
#         # this dictionary contains a subset of all the dataframe features i.e what is needed for every experiment,
#         # it is indexed by bar
#         pickle_out_returns = os.path.join(experimentsLocation,
#                                           "".join((str('FB1'), "_" + str(date_idx) + "_experiments_dict.pkl")))
#         pickle.dump(output_dict, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
#         print('saved')


In [8]:
files_idx = 10
date_idx = files[files_idx].split(".")[0]
idx_file = os.path.join(du_folder, files[files_idx]) # the input here needs to be dynamic not du_folder or rx_folder
tick_df = pd.read_pickle(idx_file)[date_idx]['tick']
volume_df = pd.read_pickle(idx_file)[date_idx]['volume']
calendar_df = pd.read_pickle(idx_file)[date_idx]['calendar']
dollar_df = pd.read_pickle(idx_file)[date_idx]['dollar']


In [16]:
for files_idx in range(0,51):
    date_idx = files[files_idx].split(".")[0]
    try:
        
        idx_file = os.path.join(du_folder, files[files_idx]) # the input here needs to be dynamic not du_folder or rx_folder
        tick_df = pd.read_pickle(idx_file)[date_idx]['tick']
        volume_df = pd.read_pickle(idx_file)[date_idx]['volume']
        calendar_df = pd.read_pickle(idx_file)[date_idx]['calendar']
        dollar_df = pd.read_pickle(idx_file)[date_idx]['dollar']
        print(produce_experiment_data(tick_df))
        print(produce_experiment_data(dollar_df))
    except ValueError:
        print("error here:", date_idx)

Applying micro-structure features
error
    micro_price  price_imbalance  pct_change_micro_price  weighted_activity_spread    GK_vol  arrival_rates  returns_normalised  returns_mix_max  skew          kurt  median_traded_volume
0    158.887353       -74.771471                0.000000                 -0.002444  0.002062            0.0            0.000000         0.000000   0.0  0.000000e+00                   1.5
1    158.547812        29.724687               -0.003462                 -0.006275  0.001999          154.0           -0.344654         0.520964   0.0  0.000000e+00                   1.0
2    158.684706        56.047059                0.002095                  0.092964  0.001873         3449.0            0.188683         0.594405   0.0  0.000000e+00                   1.0
3    158.754848        33.693636                0.000179                  0.038462  0.001867         6572.0            0.004758         0.569079   0.0  0.000000e+00                   1.0
4    158.721892       -90

/home/ak/Documents/PaperCode/stylised_facts/lob_for_futures/__init__.py:204: RuntimeWarning: invalid value encountered in sqrt
  rs = np.sqrt(self.M_parameter()[sampling_param]) * (
/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


error
      micro_price  price_imbalance  pct_change_micro_price  weighted_activity_spread  GK_vol  arrival_rates  returns_normalised  returns_mix_max  skew  kurt  median_traded_volume
0      158.725000         0.035000                0.000000                      0.07     0.0           10.0            0.000000         0.000000   0.0   0.0                   3.0
1      158.720000         0.040000               -0.000032                      0.08     0.0            6.0            0.048179         0.971050   0.0   0.0                   3.0
2      158.715000         0.045000               -0.000032                      0.09     0.0           14.0            0.048179         0.971050   0.0   0.0                   3.0
3      158.725000         0.075000                0.000063                      0.15     0.0            2.0            0.050117         0.971142   0.0   0.0                   3.0
4      158.730000         0.070000                0.000032                      0.14     0.0       

/home/ak/Documents/PaperCode/stylised_facts/lob_for_futures/__init__.py:204: RuntimeWarning: invalid value encountered in sqrt
  rs = np.sqrt(self.M_parameter()[sampling_param]) * (
/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


error
      micro_price  price_imbalance  pct_change_micro_price  weighted_activity_spread  GK_vol  arrival_rates  returns_normalised  returns_mix_max  skew  kurt  median_traded_volume
0      159.393333        53.206667                0.000000                      0.17     0.0           15.0            0.000000         0.000000   0.0   0.0                   5.0
1      159.415000         0.035000                0.000136                      0.07     0.0           14.0            0.040794         0.967018   0.0   0.0                   5.0
2      159.415000         0.025000                0.000000                      0.05     0.0          134.0            0.037132         0.966887   0.0   0.0                   5.0
3      159.426667       -53.133333                0.000073                      0.02     0.0           25.0            0.039103         0.966957   0.0   0.0                   5.0
4      159.440000       -31.864000                0.000084                      0.05     0.0       

...

error
      micro_price  price_imbalance  pct_change_micro_price  weighted_activity_spread  GK_vol  arrival_rates  returns_normalised  returns_mix_max  skew  kurt  median_traded_volume
0         158.725            0.035                0.000000                      0.07     0.0            0.0            0.000000         0.000000   0.0   0.0                   3.0
1         158.720            0.040               -0.000032                      0.08     0.0           10.0            0.050541         0.982853   0.0   0.0                   3.0
2         158.715            0.045               -0.000032                      0.09     0.0            6.0            0.050541         0.982853   0.0   0.0                   3.0
3         158.725            0.075                0.000063                      0.15     0.0           14.0            0.052389         0.982945   0.0   0.0                   3.0
4         158.730            0.070                0.000032                      0.14     0.0       

/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


     micro_price  price_imbalance  pct_change_micro_price  weighted_activity_spread  GK_vol  arrival_rates  returns_normalised  returns_mix_max  skew  kurt  median_traded_volume  jumps_test  relz_var  trip_quart
0     159.393333        53.206667                0.000000                      0.17     0.0           15.0            0.000000         0.000000   0.0   0.0                   5.0         0.0       0.0         0.0
1     159.415000         0.035000                0.000136                      0.07     0.0           14.0            0.815704         0.698058   0.0   0.0                   5.0         0.0       0.0         0.0
2     159.415000         0.025000                0.000000                      0.05     0.0          134.0           -0.025140         0.637428   0.0   0.0                   5.0         0.0       0.0         0.0
3     159.426667       -53.133333                0.000073                      0.02     0.0           25.0            0.427560         0.670071   0.0   

/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


KeyboardInterrupt: 

In [29]:
pool = Pool(3)
start_time = time.perf_counter()
processes = [pool.apply_async(produce_and_dump, args=(files_idx,)) for files_idx in range(0, 10)]
result = [p.get() for p in processes]
finish_time = time.perf_counter()
print(f"Program finished in {finish_time - start_time} seconds")
print(result)

Applying micro-structure features


/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Applying micro-structure features
Applying micro-structure features


/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Applying micro-structure features


...

error
Applying micro-structure features


/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


error
Applying micro-structure features


/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


error
Applying micro-structure features


In [17]:
# 2018-09-07 06:30:00 

In [33]:
[f for f in files if str('201809') in f]

['20180911.pkl',
 '20180905.pkl',
 '20180924.pkl',
 '20180907.pkl',
 '20180912.pkl',
 '20180927.pkl',
 '20180926.pkl',
 '20180919.pkl',
 '20180914.pkl',
 '20180920.pkl',
 '20180904.pkl',
 '20180913.pkl',
 '20180921.pkl',
 '20180918.pkl',
 '20180928.pkl',
 '20180917.pkl',
 '20180925.pkl',
 '20180906.pkl']

In [57]:
# issue dates:  '20180907.pkl',
problem_file_loc = os.path.join(du_folder, '20180417.pkl')

In [58]:
problem_dict = pd.read_pickle(problem_file_loc)['20180417']

In [60]:
tick_problem = problem_dict['tick']
dollar_problem = problem_dict['dollar']
volume_problem = problem_dict['volume']
calendar_problem = problem_dict['calendar']


In [61]:
tick_problem.head(5)

,BidSize_open,BidSize_high,BidSize_low,BidSize_close,QuoteTime_open,QuoteTime_high,QuoteTime_low,QuoteTime_close,type_x_open,type_x_high,type_x_low,type_x_close,BestBid_open,BestBid_high,BestBid_low,BestBid_close,TradeTime_open,TradeTime_high,TradeTime_low,TradeTime_close,AskSize_open,AskSize_high,AskSize_low,AskSize_close,time_y_open,time_y_high,time_y_low,time_y_close,type_y_open,type_y_high,type_y_low,type_y_close,BestAsk_open,BestAsk_high,BestAsk_low,BestAsk_close,TradeSize_open,TradeSize_high,TradeSize_low,TradeSize_close,type_open,type_high,type_low,type_close,TradePrice_open,TradePrice_high,TradePrice_low,TradePrice_close,TimeStamp_open,TimeStamp_high,TimeStamp_low,TimeStamp_close,TradeVolume_open,TradeVolume_high,TradeVolume_low,TradeVolume_close,total_traded_volume_open,total_traded_volume_high,total_traded_volume_low,total_traded_volume_close,milliSeconds_open,milliSeconds_high,milliSeconds_low,milliSeconds_close,dollar_traded_volume_open,dollar_traded_volume_high,dollar_traded_volume_low,dollar_traded_volume_close,timeStampIdx_open,timeStampIdx_high,timeStampIdx_low,timeStampIdx_close,micro_price_open,micro_price_high,micro_price_low,micro_price_close
tick_bucket,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,2070,6521,20,234,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,BID,BID,BID,BID,111.865,111.870,111.86,111.865,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,6128,6128,20,2395,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,ASK,ASK,ASK,ASK,111.875,111.875,111.865,111.870,6,500,1,221,TRADE,TRADE,TRADE,TRADE,111.870,111.870,111.865,111.865,06:01:29.168000,06:03:58.079000,06:01:29.168000,06:03:58.079000,6,500,1,221,6,500,1,221,21689000,21838000,21689000,21838000,671.220,55932.500,111.865,24722.165,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,111.872475,111.874914,111.860015,111.869555
2.0,44,7390,2,1039,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,BID,BID,BID,BID,111.865,111.865,111.86,111.865,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,3643,3643,150,150,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,ASK,ASK,ASK,ASK,111.870,111.870,111.865,111.865,91,141,17,141,TRADE,TRADE,TRADE,TRADE,111.865,111.865,111.865,111.865,06:04:10.564000,06:08:19.010000,06:04:10.564000,06:08:19.010000,91,141,17,141,91,141,17,141,21850000,22099000,21850000,22099000,10179.715,15772.965,1901.705,15772.965,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,111.869940,111.869997,111.860099,111.865000
3.0,7272,7463,4,116,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,BID,BID,BID,BID,111.860,111.865,111.86,111.865,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,150,4195,150,3546,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,ASK,ASK,ASK,ASK,111.865,111.870,111.865,111.870,141,319,1,70,TRADE,TRADE,TRADE,TRADE,111.865,111.865,111.865,111.865,06:08:19.010000,06:08:49.406000,06:08:19.010000,06:08:49.406000,141,319,1,70,141,319,1,70,22099000,22129000,22099000,22129000,15772.965,35684.935,111.865,7830.550,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,111.860101,111.869995,111.860101,111.869842
4.0,104,7482,3,632,2018-05-15 06:08:49.549,2018-05-15 06:10:27.536,2018-05-15 06:08:49.549,2018-05-15 06:10:27.536,BID,BID,BID,BID,111.865,111.865,111.86,111.865,2018-05-15 06:08:49.549,2018-05-15 06:10:27.536,2018-05-15 06:08:49.549,2018-05-15 06:10:27.536,3546,5596,97,4050,2018-05-15 06:08:48.714,2018-05-15 06:10:14.859,2018-05-15 06:08:48.714,201

In [62]:
lobfut.volatilityEstimation(tick_problem).arrival_rates()

array([[161.],
       [249.],
       [ 30.],
       ...,
       [400.],
       [371.],
       [374.]])

In [63]:
lobfut.apply_micro_structure_features(tick_problem)

Applying micro-structure features


,tick_bucket,BidSize_open,BidSize_high,BidSize_low,BidSize_close,QuoteTime_open,QuoteTime_high,QuoteTime_low,QuoteTime_close,type_x_open,type_x_high,type_x_low,type_x_close,BestBid_open,BestBid_high,BestBid_low,BestBid_close,TradeTime_open,TradeTime_high,TradeTime_low,TradeTime_close,AskSize_open,AskSize_high,AskSize_low,AskSize_close,time_y_open,time_y_high,time_y_low,time_y_close,type_y_open,type_y_high,type_y_low,type_y_close,BestAsk_open,BestAsk_high,BestAsk_low,BestAsk_close,TradeSize_open,TradeSize_high,TradeSize_low,TradeSize_close,type_open,type_high,type_low,type_close,TradePrice_open,TradePrice_high,TradePrice_low,TradePrice_close,TimeStamp_open,TimeStamp_high,TimeStamp_low,TimeStamp_close,TradeVolume_open,TradeVolume_high,TradeVolume_low,TradeVolume_close,total_traded_volume_open,total_traded_volume_high,total_traded_volume_low,total_traded_volume_close,milliSeconds_open,milliSeconds_high,milliSeconds_low,milliSeconds_close,dollar_traded_volume_open,dollar_traded_volume_high,dollar_traded_volume_low,dollar_traded_volume_close,timeStampIdx_open,timeStampIdx_high,timeStampIdx_low,timeStampIdx_close,micro_price_open,micro_price_high,micro_price_low,micro_price_close,weighted_average_BestBid,weighted_average_BestAsk,weighted_activity_spread,total_size,micro_price,price_imbalance,pct_change_micro_price,simple_mid_price
0,1.0,2070,6521,20,234,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,BID,BID,BID,BID,111.865,111.870,111.86,111.865,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,2018-05-15 06:01:29.168,2018-05-15 06:03:58.079,6128,6128,20,2395,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,ASK,ASK,ASK,ASK,111.875,111.875,111.865,111.870,6,500,1,221,TRADE,TRADE,TRADE,TRADE,111.870,111.870,111.865,111.865,06:01:29.168000,06:03:58.079000,06:01:29.168000,06:03:58.079000,6,500,1,221,6,500,1,221,21689000,21838000,21689000,21838000,671.220,55932.500,111.865,24722.165,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,2018-05-15 06:01:29.168,2018-05-15 06:03:57.360,111.872475,111.874914,111.860015,111.869555,111.868675,111.874170,0.005495,23516,111.872103,27.718469,NaN,111.868785
1,2.0,44,7390,2,1039,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,BID,BID,BID,BID,111.865,111.865,111.86,111.865,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,3643,3643,150,150,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,ASK,ASK,ASK,ASK,111.870,111.870,111.865,111.865,91,141,17,141,TRADE,TRADE,TRADE,TRADE,111.865,111.865,111.865,111.865,06:04:10.564000,06:08:19.010000,06:04:10.564000,06:08:19.010000,91,141,17,141,91,141,17,141,21850000,22099000,21850000,22099000,10179.715,15772.965,1901.705,15772.965,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,2018-05-15 06:04:10.564,2018-05-15 06:08:19.010,111.869940,111.869997,111.860099,111.865000,111.864999,111.869802,0.004803,16061,111.867268,-6.189624,-0.000041,111.865980
2,3.0,7272,7463,4,116,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,BID,BID,BID,BID,111.860,111.865,111.86,111.865,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,2018-05-15 06:08:19.010,2018-05-15 06:08:49.406,150,4195,150,3546,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,ASK,ASK,ASK,ASK,111.865,111.870,111.865,111.870,141,319,1,70,TRADE,TRADE,TRADE,TRADE,111.865,111.865,111.865,111.865,06:08:19.010000,06:08:49.406000,06:08:19.010000,06:08:49.406000,141,319,1,70,141,319,1,70,22099000,22129000,22099000,22129000,15772.965,35684.935,111.865,7830.550,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,2018-05-15 06:08:19.010,2018-05-15 06:08:48.714,111.860101,111.869995,111.860101,111.869842,111.862551,111.869813,0.007262,22896,111.865102,-33.288479,0.000043,111.865236
3,4.0,104,7482,3,632,2018-05-15 06:0